<a href="https://colab.research.google.com/github/DJCordhose/practical-llm/blob/main/GT-Compute.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Derive Ground Truth

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules # True if running in Google Colab.
IN_COLAB

True

In [2]:
if IN_COLAB:
    !pip install -q openai

In [3]:
from google.colab import userdata

import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
from openai import OpenAI

client = OpenAI()

In [5]:
llama_results = [
    "results_Lllama_3.1_8B_16bit_de.xlsx",
    "results_Lllama_3.1_8B_16bit_en.xlsx",
    "results_Lllama_3.1_8B_4bit_de.xlsx",
    "results_Lllama_3.1_8B_4bit_en.xlsx",
    "results_Lllama_3.1_8B_8bit_de.xlsx",
    "results_Lllama_3.1_8B_8bit_en.xlsx"
]

In [6]:
gpt_results = [
    "results_gpt-4-turbo_de.xlsx",
    "results_gpt-4-turbo_en.xlsx",
    "results_gpt-3.5-turbo_de.xlsx",
    "results_gpt-3.5-turbo_en.xlsx",
    "results_gpt-4o_de.xlsx",
    "results_gpt-4o_en.xlsx",
    "results_gpt-4o-mini_de.xlsx",
    "results_gpt-4o-mini_en.xlsx"
]

In [7]:
mixtral_results = [
    "results_Mixtral-8x7B_de.xlsx",
    "results_Mixtral-8x7B_en.xlsx"
]

In [8]:
phi_results = [
    "results-Phi-3.5-MoE_4bit_en.xlsx",
    "results_Phi-3.5-MoE_4bit_de.xlsx",
    "results_Phi-3.5-mini_16bit_en.xlsx",
    "results_Phi-3.5-mini_16bit_de.xlsx",
]

In [9]:
results = llama_results + gpt_results + mixtral_results + phi_results

In [10]:
import pandas as pd

base_url = "https://github.com/DJCordhose/practical-llm/raw/main/results/"
file_path = f"{base_url}/results-Phi-3.5-MoE_4bit_en.xlsx"
df1 = pd.read_excel(file_path)
df1.columns


Index(['assesment', 'y_true', 'y_hat', 'explanation'], dtype='object')

In [11]:
data_frames = []
for result in results:
    print(result)
    file_path = f"{base_url}/{result}"
    df = pd.read_excel(file_path)
    df.rename(columns={'assesment': 'assessment'}, inplace=True)
    # Create a unique column name based on the file name (or any unique identifier)
    model_name = result.replace('results_', '').replace('.xlsx', '')
    df = df.rename(columns={'explanation': model_name}).loc[:, ['assessment', model_name]]
    data_frames.append(df)

from functools import reduce
df_final = reduce(lambda left, right: pd.merge(left, right, on='assessment', how='outer'), data_frames)

results_Lllama_3.1_8B_16bit_de.xlsx
results_Lllama_3.1_8B_16bit_en.xlsx
results_Lllama_3.1_8B_4bit_de.xlsx
results_Lllama_3.1_8B_4bit_en.xlsx
results_Lllama_3.1_8B_8bit_de.xlsx
results_Lllama_3.1_8B_8bit_en.xlsx
results_gpt-4-turbo_de.xlsx
results_gpt-4-turbo_en.xlsx
results_gpt-3.5-turbo_de.xlsx
results_gpt-3.5-turbo_en.xlsx
results_gpt-4o_de.xlsx
results_gpt-4o_en.xlsx
results_gpt-4o-mini_de.xlsx
results_gpt-4o-mini_en.xlsx
results_Mixtral-8x7B_de.xlsx
results_Mixtral-8x7B_en.xlsx
results-Phi-3.5-MoE_4bit_en.xlsx
results_Phi-3.5-MoE_4bit_de.xlsx
results_Phi-3.5-mini_16bit_en.xlsx
results_Phi-3.5-mini_16bit_de.xlsx


In [12]:
df_final['combined_explanations'] = df_final.apply(lambda row: '\n---\n'.join(row.dropna()[1:].astype(str)), axis=1)

In [13]:
df_final["y_true"] = ['Negative'] * 5 + ['Positive'] * 5 + ['Negative'] * 5 + ['Positive'] * 5

In [14]:
df_final

,assessment,Lllama_3.1_8B_16bit_de,Lllama_3.1_8B_16bit_en,Lllama_3.1_8B_4bit_de,Lllama_3.1_8B_4bit_en,Lllama_3.1_8B_8bit_de,Lllama_3.1_8B_8bit_en,gpt-4-turbo_de,gpt-4-turbo_en,gpt-3.5-turbo_de,...,gpt-4o-mini_de,gpt-4o-mini_en,Mixtral-8x7B_de,Mixtral-8x7B_en,results-Phi-3.5-MoE_4bit_en,Phi-3.5-MoE_4bit_de,Phi-3.5-mini_16bit_en,Phi-3.5-mini_16bit_de,combined_explanations,y_true
0,Aus der aktuell als verordnungsbegründend bena...,"Nein, da die Diagnose nicht zu einem konkreten...",NaN,"Nein, \nDie Diagnose ist zu vage, um eine konk...",NaN,"Nein, da keine konkreten Befunde abgeleitet we...",NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,"Nein. Die fehlende Möglichkeit, aus der genann...",...,Nein. Die fehlende Ableitbarkeit eines konkret...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, eine negative Empfehlung wird gegeben. D...","Nein, da die Diagnose nicht zu einem konkreten...",Negative
1,Gemäß den Leistungsauszügen der Krankenkasse i...,Nein\n\nBegründung: Der Versicherte ist bereit...,NaN,"Nein, \nDie Einschätzung ist negativ, da der V...",NaN,Nein\n\nBegründung: Der Versicherte ist bereit...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Ja. Der Versicherte ist bereits entsprechend d...,...,"Nein. Die Bewertung zeigt, dass der Versichert...",NaN,Aufgrund der vorgelegten Leistungsauszüge der ...,NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Ja, eine positive Empfehlung wird gegeben. Die...",Nein\n\nBegründung: Der Versicherte ist bereit...,Negative
2,"Eine medizinisch nachvollziehbare Begründung, ...",Nein\n\nBegründung: Die medizinische Begründun...,NaN,"Nein\n\nDie Begründung fehlt, daher kann keine...",NaN,Nein\n\nBegründung: Die erforderliche medizini...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Nein. Es wurde keine positive Empfehlung gegeb...,...,Nein. Es wurde keine ausreichende medizinische...,NaN,"Nein, es wird keine positive Empfehlung für di...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, eine positive Empfehlung wird nicht gege...",Nein\n\nBegründung: Die medizinische Begründun...,Negative
3,In der Gesamtschau der hier vorliegenden Infor...,"Nein, da keine Begründung für die Versorgung u...",NaN,"Nein, da keine ausreichenden Informationen vor...",NaN,Nein\n\nBegründung: Die Informationen reichen ...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Nein. Die vorliegenden Informationen deuten da...,...,"Nein. Die Bewertung zeigt, dass keine ausreich...",NaN,"Nein, es wird keine positive Empfehlung hinsic...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, die Bewertung gibt keine positive Empfeh...","Nein, da keine Begründung für die Versorgung u...",Negative
4,"Eine ärztliche Begründung, warum im vorliegend...",Nein\n\nBegründung: Die Bewertung fehlt eine ä...,NaN,"Nein, \nDie Begründung fehlt, daher kann keine...",NaN,Nein\n\nBegründung: Die ärztliche Begründung f...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Nein. Eine ärztliche Begründung für den Einsat...,...,"Nein. Es liegt keine ärztliche Begründung vor,...",NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, die Bewertung deutet auf eine negative E...",Nein\n\nBegründung: Die Bewertung fehlt eine ä...,Negative
5,Bei der hier benannten Diagnose ist das Erford...,"Nein\n\nBegründung: Die Aussage besagt, dass e...",NaN,"Ja, eine negative Empfehlung wird gegeben. \n\...",NaN,"Nein\n\nBegründung: Die Aussage besagt, dass e...",NaN,"Ja, es wird eine positive Empfehlung gegeben. ...",NaN,Ja. Aufgrund der genannten Diagnose und des po...,...,"Ja. Die Bewertung deutet darauf hin, dass ein ...",NaN,"Ja, eine positive Empfehlung wird abgegeben.\n...",NaN,NaN,"Nein, es wird keine klare positive oder negati...",NaN,"Ja, eine positive Empfehlung wird gegeben. Die...","Nein\n\nBe

In [15]:
def derive_gt(explanations, gt):
  messages = [
      {"role": "system", "content": '''
Please analyze the different explanations provided and synthesize a comprehensive and conclusive ground truth explanation based on the insights.
Explanations are separated by '---'.
Ground Truth is the only source for yes/no ja/nein in the answer generated.
Answer with Yes/No Ja/Nein and directly after a single sentence.
Stick to the language of the explanations, either English or German.
  '''},
      {"role": "user", "content": f'''
  # Combined Explanations
  {explanations}
  # Ground Truth
  {gt}
      '''}
  ]

  completion = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    temperature=0.0,
  )
  return completion.choices[0].message.content

In [16]:
gts = []
for row in df_final.iterrows():
  explanations = row[1]['combined_explanations']
  y_true = row[1]['y_true']
  gt = derive_gt(explanations, y_true)
  gts.append(gt)
  print(gt)

Nein, da keine konkreten Befunde aus der Diagnose abgeleitet werden können, kann keine fundierte Empfehlung gegeben werden.
Nein. Der Versicherte ist bereits entsprechend dem Einsatzbereich des beantragten funktionellen Produkts versorgt, was bedeutet, dass die erforderliche Leistung bereits durch die Krankenkasse erbracht wurde.
Nein, eine medizinisch nachvollziehbare Begründung für die Notwendigkeit einer elektrischen Fußheberstimulation fehlt.
Nein, da keine ausreichenden Informationen vorliegen, um die Notwendigkeit und Unbedenklichkeit der Versorgung zu bestätigen.
Nein, eine ärztliche Begründung für den Einsatz eines nicht im Hilfsmittelverzeichnis gelisteten Produkts fehlt, was eine positive Empfehlung verhindert.
Ja. Die Mehrheit der Erklärungen unterstützt eine positive Empfehlung, basierend auf der Notwendigkeit eines Ausgleichs zur Sicherstellung des Grundbedürfnisses.
Ja, die sozialmedizinischen Voraussetzungen sind erfüllt, was eine positive Empfehlung für die Hilfsmittelv

In [17]:
df_final['Ground_Truth'] = gts

In [18]:
df_final

,assessment,Lllama_3.1_8B_16bit_de,Lllama_3.1_8B_16bit_en,Lllama_3.1_8B_4bit_de,Lllama_3.1_8B_4bit_en,Lllama_3.1_8B_8bit_de,Lllama_3.1_8B_8bit_en,gpt-4-turbo_de,gpt-4-turbo_en,gpt-3.5-turbo_de,...,gpt-4o-mini_en,Mixtral-8x7B_de,Mixtral-8x7B_en,results-Phi-3.5-MoE_4bit_en,Phi-3.5-MoE_4bit_de,Phi-3.5-mini_16bit_en,Phi-3.5-mini_16bit_de,combined_explanations,y_true,Ground_Truth
0,Aus der aktuell als verordnungsbegründend bena...,"Nein, da die Diagnose nicht zu einem konkreten...",NaN,"Nein, \nDie Diagnose ist zu vage, um eine konk...",NaN,"Nein, da keine konkreten Befunde abgeleitet we...",NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,"Nein. Die fehlende Möglichkeit, aus der genann...",...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, eine negative Empfehlung wird gegeben. D...","Nein, da die Diagnose nicht zu einem konkreten...",Negative,"Nein, da keine konkreten Befunde aus der Diagn..."
1,Gemäß den Leistungsauszügen der Krankenkasse i...,Nein\n\nBegründung: Der Versicherte ist bereit...,NaN,"Nein, \nDie Einschätzung ist negativ, da der V...",NaN,Nein\n\nBegründung: Der Versicherte ist bereit...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Ja. Der Versicherte ist bereits entsprechend d...,...,NaN,Aufgrund der vorgelegten Leistungsauszüge der ...,NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Ja, eine positive Empfehlung wird gegeben. Die...",Nein\n\nBegründung: Der Versicherte ist bereit...,Negative,Nein. Der Versicherte ist bereits entsprechend...
2,"Eine medizinisch nachvollziehbare Begründung, ...",Nein\n\nBegründung: Die medizinische Begründun...,NaN,"Nein\n\nDie Begründung fehlt, daher kann keine...",NaN,Nein\n\nBegründung: Die erforderliche medizini...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Nein. Es wurde keine positive Empfehlung gegeb...,...,NaN,"Nein, es wird keine positive Empfehlung für di...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, eine positive Empfehlung wird nicht gege...",Nein\n\nBegründung: Die medizinische Begründun...,Negative,"Nein, eine medizinisch nachvollziehbare Begrün..."
3,In der Gesamtschau der hier vorliegenden Infor...,"Nein, da keine Begründung für die Versorgung u...",NaN,"Nein, da keine ausreichenden Informationen vor...",NaN,Nein\n\nBegründung: Die Informationen reichen ...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Nein. Die vorliegenden Informationen deuten da...,...,NaN,"Nein, es wird keine positive Empfehlung hinsic...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, die Bewertung gibt keine positive Empfeh...","Nein, da keine Begründung für die Versorgung u...",Negative,"Nein, da keine ausreichenden Informationen vor..."
4,"Eine ärztliche Begründung, warum im vorliegend...",Nein\n\nBegründung: Die Bewertung fehlt eine ä...,NaN,"Nein, \nDie Begründung fehlt, daher kann keine...",NaN,Nein\n\nBegründung: Die ärztliche Begründung f...,NaN,"Nein, es wird eine negative Empfehlung gegeben...",NaN,Nein. Eine ärztliche Begründung für den Einsat...,...,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,NaN,"Nein, es wird keine positive Empfehlung gegebe...",NaN,"Nein, die Bewertung deutet auf eine negative E...",Nein\n\nBegründung: Die Bewertung fehlt eine ä...,Negative,"Nein, eine ärztliche Begründung für den Einsat..."
5,Bei der hier benannten Diagnose ist das Erford...,"Nein\n\nBegründung: Die Aussage besagt, dass e...",NaN,"Ja, eine negative Empfehlung wird gegeben. \n\...",NaN,"Nein\n\nBegründung: Die Aussage besagt, dass e...",NaN,"Ja, es wird eine positive Empfehlung gegeben. ...",NaN,Ja. Aufgrund der genannten Diagnose und des po...,...,NaN,"Ja, eine positive Empfehlung wird abgegeben.\n...",NaN,NaN,"Nein, es wird keine klare positive oder negati...",NaN,"Ja, eine positive Empfehlung wird gegeben. Die...","Nein\n\nBegründung: Die Aussage besagt, dass e...",Positive,Ja

In [19]:
df_final.to_excel("Results_with_Ground_Truth.xlsx", index=False)